In [117]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 


In [118]:
train__set_features = pd.read_csv("training_set_features.csv")
train__set_features.shape

train__set_labels = pd.read_csv("training_set_labels.csv")

y_h1n1 = train__set_labels['h1n1_vaccine']
y_h1n1

y_seasonal = train__set_labels['seasonal_vaccine']

train__set_features.shape

(26707, 36)

In [119]:
y_seasonal.shape

(26707,)

In [120]:
train = pd.concat([train__set_features, y_h1n1, y_seasonal], axis=1)
train.columns



Index(['respondent_id', 'h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation', 'h1n1_vaccine', 'seasonal_vaccine'],
      dtype='object')

On va gerer les valeurs manquantes et les valeurs categoriels 

In [121]:
train.select_dtypes(include='object').isna().sum()
# train = train.drop(['employment_industry'], axis=1)
# la moitié de employement _industry et _occupation est manquante 


age_group                    0
education                 1407
race                         0
sex                          0
income_poverty            4423
marital_status            1408
rent_or_own               2042
employment_status         1463
hhs_geo_region               0
census_msa                   0
employment_industry      13330
employment_occupation    13470
dtype: int64

In [122]:
train = train.drop(["employment_industry", "employment_occupation"], axis=1)

train = pd.get_dummies(train, columns=['education', 'income_poverty', 'marital_status', 'rent_or_own', 'employment_status'])

In [123]:
from sklearn.neighbors import KNeighborsRegressor
def manage_na(X,na_col): 
    X = X.copy()
    numerical = X.select_dtypes(np.number)
    non_na_columns = numerical.loc[:,numerical.isna().sum() == 0].columns
    X_train = numerical.loc[numerical[na_col].isna() == False,non_na_columns]
    y_train = numerical.loc[numerical[na_col].isna() == False,na_col]
    X_na = numerical.loc[numerical[na_col].isna() == True, non_na_columns]
    knn = KNeighborsRegressor()
    knn.fit(X_train, y_train)
    y_na = knn.predict(X_na)
    X.loc[X[na_col].isna() == True, na_col] = y_na
    
    return X 

In [124]:
cond = train.corr()['seasonal_vaccine'].apply(abs).sort_values() > 0.1

nulls = train[cond.index[cond]].isna().sum() > 0

for col in nulls[nulls == True].index:
    train = manage_na(train, col)




C:\Users\Amélie\AppData\Local\Temp\ipykernel_23752\397326579.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  cond = train.corr()['seasonal_vaccine'].apply(abs).sort_values() > 0.1


In [125]:
nulls = train.isna().sum().sort_values() > 0


for col in nulls[nulls == True].index:
    train = manage_na(train, col)

train.isna().sum()



respondent_id                               0
h1n1_concern                                0
h1n1_knowledge                              0
behavioral_antiviral_meds                   0
behavioral_avoidance                        0
behavioral_face_mask                        0
behavioral_wash_hands                       0
behavioral_large_gatherings                 0
behavioral_outside_home                     0
behavioral_touch_face                       0
doctor_recc_h1n1                            0
doctor_recc_seasonal                        0
chronic_med_condition                       0
child_under_6_months                        0
health_worker                               0
health_insurance                            0
opinion_h1n1_vacc_effective                 0
opinion_h1n1_risk                           0
opinion_h1n1_sick_from_vacc                 0
opinion_seas_vacc_effective                 0
opinion_seas_risk                           0
opinion_seas_sick_from_vacc       

In [127]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

y = train["seasonal_vaccine"]

X = train.drop("seasonal_vaccine", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, random_state=32,)

params = {
    "n_estimators": np.arange(1, 20,5), 
    "max_depth": np.arange(1, 20, 5),
    "min_samples_leaf": np.arange(1, 20, 5), 
}
grid = GridSearchCV(RandomForestClassifier(), params, cv=5)

X_train.select_dtypes(include="object")

,age_group,race,sex,hhs_geo_region,census_msa
13034,18 - 34 Years,White,Female,kbazzjca,Non-MSA
22588,18 - 34 Years,White,Female,bhuqouqj,"MSA, Not Principle City"
13549,55 - 64 Years,White,Male,oxchjgsf,Non-MSA
10539,65+ Years,Hispanic,Female,kbazzjca,"MSA, Principle City"
21586,35 - 44 Years,White,Male,fpwskwrf,Non-MSA
...,...,...,...,...,...
23955,45 - 54 Years,White,Male,qufhixun,"MSA, Not Principle City"
20414,35 - 44 Years,White,Female,mlyzmhmf,"MSA, Principle City"
9526,18 - 34 Years,Black,Male,lrircsnp,"MSA, Principle City"
9771,65+ Years,White,Female,fpwskwrf,"MSA, Not Principle City"
